In [1]:
import numpy as np
import math
import sklearn
import matplotlib.pyplot as plt
import plotly.plotly as py
import plotly.tools as tls
import pandas as pd

In [2]:
def thresh(z,delta):
    return np.sign(z)*(np.abs(z) >= delta)*(np.abs(z)-delta)

def ssvd(X, gamu = 2, gamv = 2, merr = 10**(-4), niter = 100):
    # initial values
    U, s, V = np.linalg.svd(X)
    u0 = U.T[0]
    v0 = V.T[0]
        
    n = X.shape[0]
    d = X.shape[1]
    ud = 1
    vd = 1
    iters = 0
    SST = np.sum(X*X)
    while (ud > merr or vd > merr):
        iters = iters +1
        # Updating v
        z =  X.T @ u0
        winv = np.abs(z)**gamv
        sigsq = np.abs(SST - np.sum(z*z))/(n*d-d)
        cand = z*winv
        delt = np.sort(np.append(np.abs(cand),0))
        delt_uniq = np.unique(delt)
        Bv = np.ones(len(delt_uniq)-1)*float("inf")
        ind = np.where(winv>10^(-8))
        cand1 = cand[ind]
        winv1 = winv[ind]
        for i in range(len(Bv)):
            temp2 = thresh(cand1,delta = delt_uniq[i])
            temp2 = temp2/winv1
            temp3 = np.zeros(d)
            temp3[ind] = temp2
            Bv[i] = np.sum((X - u0[:,None] @ temp3[None,:])**2)/sigsq + np.sum(temp2!=0)*math.log(n*d)
        Iv = min(np.where(Bv== np.min(Bv)))
        th = delt_uniq[Iv]
        temp2 = thresh(cand1,delta = th)
        temp2 = temp2/winv1
        v1 = np.zeros(d)
        v1[ind] = temp2
        v1 = v1/((np.sum(v1*v1))**0.5) #v_new
        
        # Updating u
        z = X @ v1
        winu = np.abs(z)**gamu
        sigsq = np.abs(SST - np.sum(z*z))/(n*d-n)
        cand = z*winu
        delt = np.sort(np.append(np.abs(cand),0))
        delt_uniq = np.unique(delt)
        Bu = np.ones(len(delt_uniq)-1)*float("inf")
        ind = np.where(winu > 10^(-8))
        cand1 = cand[ind]
        winu1 = winu[ind]
        for i in range(len(Bu)):
            temp2 = thresh(cand1,delta = delt_uniq[i])
            temp2 = temp2/winu1
            temp3 = np.zeros(n)
            temp3[ind] = temp2
            Bu[i] = np.sum((X - temp3[:,None] @ v1[None,:])**2)/sigsq + np.sum(temp2!=0)*math.log(n*d)
        Iu = min(np.where(Bu==np.min(Bu)))
        th = delt_uniq[Iu]
        temp2 = thresh(cand1,delta = th)
        temp2 = temp2/winu1
        u1 = np.zeros(n)
        u1[ind] =  temp2
        u1 = u1/((np.sum(u1*u1))**0.5)
        
        
        ud = np.sum((u0-u1)*(u0-u1))**0.5
        vd = np.sum((v0-v1)*(v0-v1))**0.5

        if iters > niter :
            print("Fail to converge! Increase the niter!")
            break
        
        u0 = u1
        v0 = v1

    s = u1[None, :] @ X @ v1[:, None]
    return u1, v1, s, iters

Simulation study Case 1 Rand 1 Approximation

In [3]:
# create u_hat
u_hat = np.array([10,9,8,7,6,5,4,3])
u_hat = np.append(u_hat,np.repeat(2,17))
u_hat = np.append(u_hat,np.repeat(0,75))

In [4]:
# create v_hat
v_hat = np.array([10,-10,8,-8,5,-5])
v_hat = np.append(v_hat,np.repeat(3,5))
v_hat = np.append(v_hat,np.repeat(-3,5))
v_hat = np.append(v_hat,np.repeat(0,34))

In [5]:
# normalize u_hat
u = u_hat/np.linalg.norm(u_hat)

In [6]:
# normalize v_hat
v = v_hat/np.linalg.norm(v_hat)

In [7]:
# create vectors for storing answers
num_zeros_u = np.zeros(100)
prop_zeros_u_identified= np.zeros(100)
prop_non_zeros_u_identified= np.zeros(100)
missclass_u= np.zeros(100)

In [8]:
#create vectors for storing answers
num_zeros_v = np.zeros(100)
prop_zeros_v_identified= np.zeros(100)
prop_non_zeros_v_identified= np.zeros(100)
missclass_v= np.zeros(100)

In [9]:
# generate X* simulated datasets.
X_star = 50*u.reshape(len(u),1) @ v.reshape(1,len(v))

In [10]:
# calculate the SVD of the matrix
U, s, V = np.linalg.svd(X_star)

SSVD

In [11]:
## simulate 100 times to generate X = X*+ epilson (sampled from standard normal distribution)
for i in range(100):
    X = X_star + np.random.normal(0,1,100*50).reshape(100,50)
    U, s, V = np.linalg.svd(X)
    u0 = U.T[0]
    v0 = V.T[0]
    Upred, Vpred ,s,niter = ssvd(X)
    ## store answers
    num_zeros_u[i] = np.sum(Upred==0)
    num_zeros_v[i] = np.sum(Vpred==0)
    prop_zeros_u_identified[i] = np.sum(Upred[np.where(u == 0)] ==0)*100/np.sum(u==0)
    prop_zeros_v_identified[i] = np.sum(Vpred[np.where(v == 0)] ==0)*100/np.sum(v==0)
    prop_non_zeros_u_identified[i] = np.sum(Upred[np.where(u != 0)]!=0)*100/np.sum(u!=0)
    prop_non_zeros_v_identified[i] = np.sum(Vpred[np.where(v != 0)]!=0)*100/np.sum(v!=0)
    missclass_u[i] = sum(np.sign(u) != np.sign(Upred)*(-1))*100/100
    missclass_v[i] = sum(np.sign(v) != np.sign(Vpred)*(-1))*100/50

In [12]:
# calculate the mean
np.mean(num_zeros_u)

74.510000000000005

In [13]:
np.mean(num_zeros_v)

33.789999999999999

In [14]:
np.mean(prop_zeros_u_identified)

98.826666666666682

In [15]:
np.mean(prop_zeros_v_identified)

99.382352941176464

In [16]:
np.mean(prop_non_zeros_u_identified)

98.439999999999998

In [17]:
np.mean(prop_non_zeros_v_identified)

100.0

In [18]:
np.mean( missclass_u)

1.27

In [19]:
np.mean( missclass_v)

0.41999999999999998

SVD 

In [20]:
# create empty vectors for storing answers.
num_zeros_u_svd = np.zeros(100)
prop_zeros_u_identified_svd= np.zeros(100)
prop_non_zeros_u_identified_svd= np.zeros(100)
missclass_u_svd= np.zeros(100)

In [21]:
# create empty vectors for storing answers.
num_zeros_v_svd = np.zeros(100)
prop_zeros_v_identified_svd= np.zeros(100)
prop_non_zeros_v_identified_svd= np.zeros(100)
missclass_v_svd= np.zeros(100)

In [22]:
for i in range(100):
    # generate X = X* + epislon (sampled from the standard normal distribution)
    X = X_star + np.random.normal(0,1,100*50).reshape(100,50)
    Upred, s, Vpred = np.linalg.svd(X)
    Upred = Upred.T[0]
    Vpred = Vpred.T[0]
    # storing all answers
    num_zeros_u_svd[i] = np.sum(Upred==0)
    num_zeros_v_svd[i] = np.sum(Vpred==0)
    prop_zeros_u_identified_svd[i] = np.sum(Upred[np.where(u == 0)] ==0)*100/np.sum(u==0)
    prop_zeros_v_identified_svd[i] = np.sum(Vpred[np.where(v == 0)] ==0)*100/np.sum(v==0)
    prop_non_zeros_u_identified_svd[i] = np.sum(Upred[np.where(u != 0)]!=0)*100/np.sum(u!=0)
    prop_non_zeros_v_identified_svd[i] = np.sum(Vpred[np.where(v != 0)]!=0)*100/np.sum(v!=0)
    missclass_u_svd[i] = sum(np.sign(u) != np.sign(Upred)*(-1))*100/100
    missclass_v_svd[i] = sum(np.sign(v) != np.sign(Vpred)*(-1))*100/50

In [23]:
# calculate the mean
np.mean( num_zeros_u_svd)

0.0

In [24]:
np.mean(num_zeros_v_svd)

0.0

In [25]:
np.mean( prop_zeros_u_identified_svd)

0.0

In [26]:
np.mean(prop_zeros_v_identified_svd)

0.0

In [27]:
np.mean(prop_non_zeros_u_identified_svd)

100.0

In [28]:
np.mean(prop_non_zeros_v_identified_svd)

100.0

In [29]:
np.mean(missclass_u_svd)

75.0

In [30]:
np.mean(missclass_v_svd)

82.260000000000005

Higher Rank Approximation (Rank 2)

In [31]:
u_hat = np.array(np.repeat(20,2))
u_hat = np.append(u_hat,np.repeat(10,4))
u_hat = np.append(u_hat,np.repeat(3,8))
u_hat = np.append(u_hat,np.repeat(1,16))
u_hat = np.append(u_hat,np.repeat(0,70))

In [32]:
v_hat = np.array(np.repeat(1,20))
v_hat = np.append(v_hat,np.repeat(0,30))

In [33]:
u_hat_2 = np.array(np.repeat(0,6))
u_hat_2 = np.append(u_hat_2,np.array([5,-5]))
u_hat_2 = np.append(u_hat_2,np.repeat(0,6))
u_hat_2 = np.append(u_hat_2,np.repeat(10,4))
u_hat_2 = np.append(u_hat_2,np.repeat(-10,4))
u_hat_2 = np.append(u_hat_2,np.repeat(0,8))
u_hat_2 = np.append(u_hat_2,np.repeat(30,6))
u_hat_2 = np.append(u_hat_2,np.repeat(0,64))

In [34]:
v_hat_2 = np.array(np.repeat(0,10))
v_hat_2 = np.append(v_hat_2,np.repeat(1,5))
v_hat_2 = np.append(v_hat_2,np.repeat(-1,5))
v_hat_2 = np.append(v_hat_2,np.repeat(0,30))

In [35]:
u = u_hat/np.linalg.norm(u_hat)
v = v_hat/np.linalg.norm(v_hat)
u2 = u_hat_2/np.linalg.norm(u_hat_2)
v2 = v_hat_2/np.linalg.norm(v_hat_2)

In [36]:
X_star = 1000*u.reshape(len(u),1) @ v.reshape(1,len(v)) + 100 * u2.reshape(len(u2),1) @ v2.reshape(1,len(v2))

In [37]:
## v and v2 are perpendicular. 
v @ v2

0.0

In [38]:
u @ u2

0.0

In [39]:
# create empty vectors for storing answers
num_zeros_u1 = np.zeros(100)
prop_zeros_u1_identified= np.zeros(100)
prop_non_zeros_u1_identified= np.zeros(100)
missclass_u1= np.zeros(100)

In [40]:
num_zeros_v1 = np.zeros(100)
prop_zeros_v1_identified= np.zeros(100)
prop_non_zeros_v1_identified= np.zeros(100)
missclass_v1= np.zeros(100)

In [41]:
num_zeros_u2 = np.zeros(100)
prop_zeros_u2_identified= np.zeros(100)
prop_non_zeros_u2_identified= np.zeros(100)
missclass_u2= np.zeros(100)

In [42]:
num_zeros_v2 = np.zeros(100)
prop_zeros_v2_identified= np.zeros(100)
prop_non_zeros_v2_identified= np.zeros(100)
missclass_v2= np.zeros(100)

In [43]:
for i in range(100):
    # generate X = X* + epsilon (sample from the standard normal distribution)
    X = X_star + np.random.normal(0,1,100*50).reshape(100,50)
    U, s, V = np.linalg.svd(X)
    u0 = U.T[0]
    v0 = V.T[0]
    Upred, Vpred,s1,niter = ssvd(X)
    residuals = X - 1000*Upred.reshape(len(Upred),1) @ Vpred.reshape(1,len(Vpred))
    U, s, V = np.linalg.svd(residuals)
    u0 = U.T[0]
    v0 = V.T[0]
    ## storing answers
    Upred_2, Vpred_2,s2, niter_2 = ssvd(residuals)
    num_zeros_u1[i] = np.sum(Upred==0)
    num_zeros_u2[i] = np.sum(Upred_2==0)
    num_zeros_v1[i] = np.sum(Vpred==0)
    num_zeros_v2[i] = np.sum(Vpred_2==0)
    prop_zeros_u1_identified[i] = np.sum(Upred[np.where(u == 0)] ==0)*100/np.sum(u==0)
    prop_zeros_u2_identified[i] = np.sum(Upred_2[np.where(u2 == 0)] ==0)*100/np.sum(u2==0)
    prop_zeros_v1_identified[i] = np.sum(Vpred[np.where(v == 0)] ==0)*100/np.sum(v==0)
    prop_zeros_v2_identified[i] = np.sum(Vpred_2[np.where(v2 == 0)] ==0)*100/np.sum(v2==0)
    prop_non_zeros_u1_identified[i] = np.sum(Upred[np.where(u != 0)]!=0)*100/np.sum(u!=0)
    prop_non_zeros_u2_identified[i] = np.sum(Upred_2[np.where(u2 != 0)]!=0)*100/np.sum(u2!=0)
    prop_non_zeros_v1_identified[i] = np.sum(Vpred[np.where(v != 0)]!=0)*100/np.sum(v!=0)
    prop_non_zeros_v2_identified[i] = np.sum(Vpred_2[np.where(v2 != 0)]!=0)*100/np.sum(v2!=0)
    missclass_u1[i] = sum(np.sign(u) != np.sign(Upred)*(-1))*100/100
    missclass_u2[i] = sum(np.sign(u2) != np.sign(Upred_2)*(-1))*100/100
    missclass_v1[i] = sum(np.sign(v) != np.sign(Vpred)*(-1))*100/50
    missclass_v2[i] = sum(np.sign(v2) != np.sign(Vpred_2)*(-1))*100/50

In [44]:
np.mean(num_zeros_u1)

70.0

In [45]:
np.mean(num_zeros_u2)

83.909999999999997

In [46]:
np.mean(num_zeros_v1)

30.0

In [47]:
np.mean(num_zeros_v2)

39.880000000000003

In [48]:
np.mean(prop_zeros_u1_identified)

100.0

In [49]:
np.mean(prop_zeros_u2_identified)

99.88095238095238

In [50]:
np.mean(prop_zeros_v1_identified)

100.0

In [51]:
np.mean(prop_zeros_v2_identified)

99.700000000000003

In [52]:
np.mean(prop_non_zeros_u1_identified)

100.0

In [53]:
np.mean(prop_non_zeros_u2_identified)

99.9375

In [54]:
np.mean(prop_non_zeros_v1_identified)

100.0

In [55]:
np.mean(prop_non_zeros_v2_identified)

100.0

In [56]:
np.mean(missclass_u1)

0.0

In [57]:
np.mean(missclass_u2)

8.0999999999999996

In [58]:
np.mean(missclass_v1)

0.0

In [59]:
np.mean(missclass_v2)

10.24

SVD

In [60]:
num_zeros_u1_svd = np.zeros(100)
prop_zeros_u1_identified_svd= np.zeros(100)
prop_non_zeros_u1_identified_svd= np.zeros(100)
missclass_u1_svd= np.zeros(100)

In [61]:
num_zeros_u2_svd = np.zeros(100)
prop_zeros_u2_identified_svd= np.zeros(100)
prop_non_zeros_u2_identified_svd= np.zeros(100)
missclass_u2_svd= np.zeros(100)

In [62]:
num_zeros_v1_svd = np.zeros(100)
prop_zeros_v1_identified_svd= np.zeros(100)
prop_non_zeros_v1_identified_svd= np.zeros(100)
missclass_v1_svd= np.zeros(100)

In [63]:
num_zeros_v2_svd = np.zeros(100)
prop_zeros_v2_identified_svd= np.zeros(100)
prop_non_zeros_v2_identified_svd= np.zeros(100)
missclass_v2_svd= np.zeros(100)

In [64]:
for i in range(100):
    X = X_star + np.random.normal(0,1,100*50).reshape(100,50)
    Upred, s, Vpred = np.linalg.svd(X)
    Upred = Upred.T[0]
    Vpred = Vpred.T[0]
    residuals = X - 1000*Upred.reshape(len(Upred),1) @ Vpred.reshape(1,len(Vpred))
    Upred_2, s2, Vpred_2 = np.linalg.svd(X)
    Upred_2 = Upred_2.T[0]
    Vpred_2 = Vpred_2.T[0]
    num_zeros_u1_svd[i] = np.sum(Upred==0)
    num_zeros_u2_svd[i] = np.sum(Upred_2==0)
    num_zeros_v1_svd[i] = np.sum(Vpred==0)
    num_zeros_v2_svd[i] = np.sum(Vpred_2==0)
    prop_zeros_u1_identified_svd[i] = np.sum(Upred[np.where(u == 0)] ==0)*100/np.sum(u==0)
    prop_zeros_u2_identified_svd[i] = np.sum(Upred_2[np.where(u2 == 0)] ==0)*100/np.sum(u2==0)
    prop_zeros_v1_identified_svd[i] = np.sum(Vpred[np.where(v == 0)] ==0)*100/np.sum(v==0)
    prop_zeros_v2_identified_svd[i] = np.sum(Vpred_2[np.where(v2 == 0)] ==0)*100/np.sum(v2==0)
    prop_non_zeros_u1_identified_svd[i] = np.sum(Upred[np.where(u != 0)]!=0)*100/np.sum(u!=0)
    prop_non_zeros_u2_identified_svd[i] = np.sum(Upred_2[np.where(u2 != 0)]!=0)*100/np.sum(u2!=0)
    prop_non_zeros_v1_identified_svd[i] = np.sum(Vpred[np.where(v != 0)]!=0)*100/np.sum(v!=0)
    prop_non_zeros_v2_identified_svd[i] = np.sum(Vpred_2[np.where(v2 != 0)]!=0)*100/np.sum(v2!=0)
    missclass_u1_svd[i] = sum(np.sign(u) != np.sign(Upred)*(-1))*100/100
    missclass_u2_svd[i] = sum(np.sign(u2) != np.sign(Upred_2)*(-1))*100/100
    missclass_v1_svd[i] = sum(np.sign(v) != np.sign(Vpred)*(-1))*100/50
    missclass_v2_svd[i] = sum(np.sign(v2) != np.sign(Vpred_2)*(-1))*100/50

In [65]:
np.mean(num_zeros_u1_svd)

0.0

In [66]:
np.mean(num_zeros_u2_svd)

0.0

In [67]:
np.mean(num_zeros_v1_svd)

0.0

In [68]:
np.mean(num_zeros_v2_svd)

0.0

In [69]:
np.mean(prop_zeros_u1_identified_svd)

0.0

In [70]:
np.mean(prop_zeros_u2_identified_svd)

0.0

In [71]:
np.mean(prop_zeros_v1_identified_svd)

0.0

In [72]:
np.mean(prop_zeros_v2_identified_svd)

0.0

In [73]:
np.mean(prop_non_zeros_u1_identified_svd)

100.0

In [74]:
np.mean(prop_non_zeros_u2_identified_svd)

100.0

In [75]:
np.mean(prop_non_zeros_v1_identified_svd)

100.0

In [76]:

np.mean(prop_non_zeros_v2_identified_svd)

100.0

In [77]:
np.mean(missclass_u1_svd)

70.0

In [78]:
np.mean(missclass_u2_svd)

92.170000000000002

In [79]:
np.mean(missclass_v1_svd)

78.540000000000006

In [80]:
np.mean(missclass_v2_svd)

89.939999999999998

In [81]:
import prettytable
from prettytable import PrettyTable

In [82]:
t = PrettyTable(['Method',"", 'Avg.# of zeros',"Avg.% of correct zeros","Avg.% of correct nonzeros","Misclassification error(%)"])
t.add_row(['SSVD', "u",np.mean(num_zeros_u),np.round(np.mean(prop_zeros_u_identified),2),np.round(np.mean(prop_non_zeros_u_identified),2),np.round(np.mean(missclass_u),2)])
t.add_row(['', "v",np.mean(num_zeros_v),np.round(np.mean(prop_zeros_v_identified),2),np.round(np.mean(prop_non_zeros_v_identified),2),np.round(np.mean(missclass_v),2)])
t.add_row(['SVD', "u",np.mean(num_zeros_u_svd),np.round(np.mean(prop_zeros_u_identified_svd),2),np.round(np.mean(prop_non_zeros_u_identified_svd),2),np.round(np.mean(missclass_u_svd),2)])
t.add_row(['', "v",np.mean(num_zeros_v_svd),np.round(np.mean(prop_zeros_v_identified_svd),2),np.round(np.mean(prop_non_zeros_v_identified_svd),2),np.round(np.mean(missclass_v_svd),2)])
print(t)

+--------+---+----------------+------------------------+---------------------------+----------------------------+
| Method |   | Avg.# of zeros | Avg.% of correct zeros | Avg.% of correct nonzeros | Misclassification error(%) |
+--------+---+----------------+------------------------+---------------------------+----------------------------+
|  SSVD  | u |     74.51      |         98.83          |           98.44           |            1.27            |
|        | v |     33.79      |         99.38          |           100.0           |            0.42            |
|  SVD   | u |      0.0       |          0.0           |           100.0           |            75.0            |
|        | v |      0.0       |          0.0           |           100.0           |           82.26            |
+--------+---+----------------+------------------------+---------------------------+----------------------------+


In [83]:
np.round(np.mean(prop_zeros_u_identified),2)

98.829999999999998

In [84]:
t = PrettyTable(['Method',"", 'Avg.# of zeros',"Avg.% of correct zeros","Avg.% of correct nonzeros","Misclassification error(%)"])
t.add_row(['SSVD', "u1",np.mean(num_zeros_u1),np.round(np.mean(prop_zeros_u1_identified),2),np.round(np.mean(prop_non_zeros_u1_identified),2),np.round(np.mean(missclass_u1),2)])
t.add_row(['', "v1",np.mean(num_zeros_v1),np.round(np.mean(prop_zeros_v1_identified),2),np.round(np.mean(prop_non_zeros_v1_identified),2),np.round(np.mean(missclass_v1),2)])
t.add_row(['', "u2",np.mean(num_zeros_u2),np.round(np.mean(prop_zeros_u2_identified),2),np.round(np.mean(prop_non_zeros_u2_identified),2),np.round(np.mean(missclass_u2),2)])
t.add_row(['', "v2",np.mean(num_zeros_v2),np.round(np.mean(prop_zeros_v2_identified),2),np.round(np.mean(prop_non_zeros_v2_identified),2),np.round(np.mean(missclass_v2),2)])
t.add_row(['SVD', "u1",np.mean(num_zeros_u1_svd),np.round(np.mean(prop_zeros_u1_identified_svd),2),np.round(np.mean(prop_non_zeros_u1_identified_svd),2),np.round(np.mean(missclass_u1_svd),2)])
t.add_row(['', "v1",np.mean(num_zeros_v1_svd),np.round(np.mean(prop_zeros_v1_identified_svd),2),np.round(np.mean(prop_non_zeros_v1_identified_svd),2),np.round(np.mean(missclass_v1_svd),2)])
t.add_row(['', "u2",np.mean(num_zeros_u2_svd),np.round(np.mean(prop_zeros_u2_identified_svd),2),np.round(np.mean(prop_non_zeros_u2_identified_svd),2),np.round(np.mean(missclass_u2_svd),2)])
t.add_row(['', "v2",np.mean(num_zeros_v2_svd),np.round(np.mean(prop_zeros_v2_identified_svd),2),np.round(np.mean(prop_non_zeros_v2_identified_svd),2),np.round(np.mean(missclass_v2_svd),2)])
print(t)

+--------+----+----------------+------------------------+---------------------------+----------------------------+
| Method |    | Avg.# of zeros | Avg.% of correct zeros | Avg.% of correct nonzeros | Misclassification error(%) |
+--------+----+----------------+------------------------+---------------------------+----------------------------+
|  SSVD  | u1 |      70.0      |         100.0          |           100.0           |            0.0             |
|        | v1 |      30.0      |         100.0          |           100.0           |            0.0             |
|        | u2 |     83.91      |         99.88          |           99.94           |            8.1             |
|        | v2 |     39.88      |          99.7          |           100.0           |           10.24            |
|  SVD   | u1 |      0.0       |          0.0           |           100.0           |            70.0            |
|        | v1 |      0.0       |          0.0           |           100.0       